<a href="https://colab.research.google.com/github/nurfnick/DataProjects/blob/main/LucyTalksData/ProjectPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Some SQL with BigQuery

## Part 1

I am going to try to follow along with **Lucy Talks Data** and do her SQL project [here](https://lucytalksdata.com/sql-for-data-science-quest-1/).  The first thing I need to do is get colab working with my BiqQuery account.  I have done all of this before by following google's suggestions on how to do it.  I am able to do this through a free account.  First I'll get logged in.

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Now that I am authenticated, I can start to play around in the dataset.  I am going to look at the liquor sales data from Iowa and try to find the most and least sales by city.

In [ ]:
%%bigquery --project pic-math
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.iowa_liquor_sales.sales`

,total_rows
0,22494544


We see that is a lot of rows.  We really don't want to try to store that into memory!  Let's have a peak at the data.

In [ ]:
%%bigquery --project pic-math
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-20825400070,2019-07-24,2555,Hy-Vee Food Store / Keokuk,3111 Main,Keokuk,52632.0,POINT (-91.403752 40.41538),56,LEE,None,None,55,SAZERAC NORTH AMERICA,84197,99 Pineapple Mini,10,600,6.00,9.00,10,90.00,6.00,1.58
1,INV-33554300013,2021-01-18,4493,Casey's General Store #2850 / Cedar Rapids,5655 C St SW,Cedar Rapids,52404.0,POINT (-91.637306 41.922505),57,LINN,1081100.0,Coffee Liqueurs,370.0,PERNOD RICARD USA,67524,Kahlua Coffee,24,375,6.49,9.74,5,48.70,1.87,0.49
2,INV-17816400003,2019-02-27,4069,Fareway Stores #648 / Ottumwa,1325 Albia Rd,Ottumwa,52501,POINT (-92.440213 41.00934),90,WAPELLO,1901200.0,Special Order Items,259.0,Heaven Hill Brands,917914,HA Elijah Craig Cask Barrel Proof,3,750,29.92,44.88,9,403.92,6.75,1.78
3,INV-23344500010,2019-11-20,4324,Dayton Community Grocery,22 North Main,Dayton,50530.0,POINT (-94.068439 42.26168),94,WEBSTER,1081100.0,Coffee Liqueurs,370.0,PERNOD RICARD USA,67527,Kahlua Coffee,12,1000,15.15,22.73,5,113.65,5.00,1.32
4,S09220000195,2012-11-29,2560,Hy-Vee Food Store / Marion,3600 BUSINESS HWY 151 EAST,MARION,52302,None,57,Linn,1081350.0,SPEARMINT SCHNAPPS,434,Luxco-St Louis,82187,Arrow Spearmint Schnapps,12,1000,4.75,7.13,4,28.52,4.00,1.06


In [ ]:
%%bigquery --project pic-math df
SELECT 
  city, 
  store_name,
  SUM(sale_dollars) as total_sales
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE city is not null
GROUP BY city, store_name

In [ ]:
df

,city,store_name,total_sales
0,Keokuk,Hy-Vee Food Store / Keokuk,4226546.86
1,Cedar Rapids,Casey's General Store #2850 / Cedar Rapids,277130.53
2,Ottumwa,Fareway Stores #648 / Ottumwa,1853189.35
3,Dayton,Dayton Community Grocery,312445.18
4,MARION,Hy-Vee Food Store / Marion,4305523.82
...,...,...,...
4144,Cedar Falls,Happy's Wine & Spirits Wholesale,1550.99
4145,Bettendorf,"Cats Eye Distillery, LLC",693.12
4146,Colorado Springs,Dash Events LLC,31.14
4147,Keosauqua,Honey Creek Distillery,235.68


In [ ]:
import pandas as pd

groupeddf = df.groupby('city')

In [ ]:
maxdf = groupeddf.max()

In [ ]:
maxdf

,store_name,total_sales
city,,
ACKLEY,KUM & GO #513 / ACKLEY,132551.63
ADAIR,KUM & GO #76 / ADAIR,148812.43
ADEL,Shugar's Super Valu / Adel,982611.78
AFTON,Casey's General Store #2626 / Afton,137832.15
AKRON,Casey's General Store #2425 / Akron,134999.66
...,...,...
Woodbine,Foodland Super Markets / Woodbine,385599.54
Woodward,Casey's General Store #2551 / Woodward,186083.15
Wyoming,Casey's General Store #1126 / Wyoming,35499.38


In [ ]:
mindf = groupeddf.min()

mindf

,store_name,total_sales
city,,
ACKLEY,Ackley Super Foods,108359.95
ADAIR,Casey's General Store #2521 / Adair,40784.57
ADEL,Casey's General Store #1680 / Adel,19733.51
AFTON,Casey's General Store #2626 / Afton,137832.15
AKRON,Casey's General Store #2425 / Akron,134999.66
...,...,...
Woodbine,Casey's General Store #2665 / Woodbine,6439.33
Woodward,Casey's General Store #2551 / Woodwa,9095.21
Wyoming,Casey's General Store #1126 / Wyoming,35499.38


I notice a few things attempting this.  While I think I have a solution, it is clearly not the best.  Zwingle and ZWINGLE are probably the same town and SNK may just be the only store but the fact that it appears four times in my lists is disappointing!

In [ ]:
maxdf.sort_values('total_sales',ascending=False)

,store_name,total_sales
city,,
Des Moines,Walgreens #07968 / Des Moines,5.785247e+07
DES MOINES,Walgreens #07968 / Des Moines,4.014780e+07
Iowa City,goPuff / Iowa City,2.576451e+07
Cedar Rapids,"Williams Boulevard Service, Inc.",1.883548e+07
Mount Vernon,Wilkie Liquors,1.794630e+07
...,...,...
RUNNELLS,Casey's General Store #2683 / Runn,5.495630e+03
GILBERTVILLE,Casey's General Store #2900 / Gilber,4.947320e+03
Malcom,Malcom Gas & Food / Malcom,4.493810e+03


I want to try the extra challenge **and** do it all in SQL.  I'll try to find which store had the most sales by date!

In [ ]:
%%bigquery --project pic-math

WITH bestday as (
SELECT 
  date, 
  store_name,
  city,
  SUM(sale_dollars) as total_sales,
  RANK() over (PARTITION BY date ORDER BY SUM(sale_dollars) desc) as top_sales_rank
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE city is not null
GROUP BY date, store_name, city
)

SELECT
  date,
  store_name,
  city,
  total_sales
FROM bestday
WHERE top_sales_rank = 1
ORDER BY date

,date,store_name,city,total_sales
0,2012-01-03,Sam's Club 8238 / Davenport,DAVENPORT,51350.70
1,2012-01-04,Hy-Vee Food Store / Sheldon,SHELDON,73296.69
2,2012-01-05,Hy-Vee #3 / BDI / Des Moines,DES MOINES,74085.23
3,2012-01-09,"Central City Liquor, Inc.",DES MOINES,60435.09
4,2012-01-10,Sam's Club 8238 / Davenport,DAVENPORT,33423.60
...,...,...,...,...
2443,2021-10-26,Costco Wholesale #788 / WDM,West Des Moines,158640.30
2444,2021-10-27,I-80 Liquor / Council Bluffs,Council Bluffs,89184.13
2445,2021-10-28,Central City 2,Des Moines,106029.76
2446,2021-10-29,Foundry Distilling Company,West Des Moines,55950.00


## Part 2

In the next assignment, I am to look at the GitHub dataset found on BigQuery and compute the average number of stars and total number of forks by programming language.  Let's get started by grabbing just the column names.

In [8]:
%%bigquery --project pic-math

SELECT *
FROM `bigquery-public-data.libraries_io.repositories`
LIMIT 10

,id,host_type,name_with_owner,description,fork,created_timestamp,updated_timestamp,last_pushed_timestamp,homepage_url,size,stars_count,language,issues_enabled,wiki_enabled,pages_enabled,forks_count,mirror_url,open_issues_count,default_branch,watchers_count,uuid,fork_source_name_with_owner,license,contributors_count,readme_filename,changelog_filename,contributing_guidelines_filename,license_filename,code_of_conduct_filename,security_threat_model_filename,security_audit_filename,status,last_synced_timestamp,sourcerank,display_name,scm_type,pull_requests_enabled,logo_url,keywords,an


This is not the cleanest way to do it.  I am clearly missing some info (like what the columns contain) but it does the job to see what is there.

In [16]:
%%bigquery --project pic-math

SELECT 
  language,
  AVG(stars_count) as avg_stars,
  SUM(forks_count) as total_forks,
  COUNT(language) as repo_count
FROM `bigquery-public-data.libraries_io.repositories`
WHERE host_type = 'GitHub' and fork is false
GROUP BY language
ORDER BY avg_stars desc
LIMIT 10



,language,avg_stars,total_forks,repo_count
0,Rascal,309.661765,10461,68
1,WebAssembly,153.500000,1448,60
2,Ragel,107.500000,115,4
3,Objective-C++,39.991508,13282,2002
4,wisp,39.272727,63,22
5,Red,38.402174,392,92
6,WebIDL,32.875000,16,8
7,Self,31.214286,137,14
8,Objective-J,29.547771,1861,471
9,Hack,28.644068,1747,295


Let's examine what the table would look like if I extract only popular languages, say 500 or more repos.

In [18]:
%%bigquery --project pic-math

WITH avg_stars_table as(
SELECT 
  language,
  AVG(stars_count) as avg_stars,
  SUM(forks_count) as total_forks,
  COUNT(language) as repo_count
FROM `bigquery-public-data.libraries_io.repositories`
WHERE host_type = 'GitHub' and fork is false and language is not null
GROUP BY language
ORDER BY avg_stars desc
)

SELECT *
FROM avg_stars_table
WHERE repo_count > 500
ORDER BY repo_count desc


,language,avg_stars,total_forks,repo_count
0,,0.545469,2385017,13379319
1,JavaScript,8.483218,5614197,2800652
2,Java,4.250502,3566884,2151703
3,Python,6.778394,2790887,1352296
4,HTML,2.086989,1293750,1287835
...,...,...,...,...
142,Squirrel,1.293561,456,528
143,Lex,3.335238,525,525
144,AGS Script,4.476923,969,520
145,xBase,0.957031,333,512
